In [ ]:
%reload_ext autoreload
%autoreload 2
from transformers import GPT2Model, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 with eager attention
model = GPT2Model.from_pretrained("gpt2", attn_implementation="eager")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize input
text = "Hello world"
inputs = tokenizer(text, return_tensors="pt")  # returns input_ids and attention_mask
# input_ids shape: (B, T) → (1, 2)

# STEP 1: Embed tokens (wte) directly without manually adding position_ids
input_ids = inputs["input_ids"]  # (B, T) → (1, 2)

# Get token embeddings (wte) and position embeddings (wpe)
input_embeds = model.wte(input_ids)  # (B, T, D) → (1, 2, 768)

# Get hidden states with added position embeddings
hidden_states = input_embeds  # Position embedding handling is implicit

# STEP 2: Run through LayerNorm + Self-Attention from the first block
first_block = model.h[0]

# Apply layer norm before attention
normed_hidden = first_block.ln_1(hidden_states)  # (B, T, D) → (1, 2, 768)

# Run self-attention
attn_output = first_block.attn(normed_hidden, head_mask=None, output_attentions=True)
attn_output = attn_output[0]  # (B, T, D) → (1, 2, 768)
# attn_output shape: (B, T, D) → (1, 2, 768)

# Now you have the output just after self-attention!
print("Attention Output Shape:", attn_output.shape)  # (1, 2, 768)

# Example: you can take just the last token’s output if you're doing next-token prediction
last_token = attn_output[:, -1, :]  # (B, D) → (1, 768)

# Plug this into your custom head
custom_output = torch.nn.Linear(768, 42)(last_token)  # Example head: projecting to 42 classes


Attention Output Shape: torch.Size([1, 2, 768])


In [7]:
model.state_dict().keys()

odict_keys(['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight', 'h.3.ln_1.bias', 'h.3.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.3.attn.c_proj.weight', 'h.3.attn.c_proj.bias', 'h.3.ln_2.weight', 'h.3.ln_2.bia

In [4]:
%reload_ext autoreload
%autoreload 2
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config
from adic_components.prototype2 import P2GPTBlock
# Load GPT-2 tokenizer and base model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model_pretrained = GPT2Model.from_pretrained('gpt2')

# Get model config to know vocab size and hidden size
config = GPT2Config.from_pretrained('gpt2')
vocab_size = config.vocab_size
hidden_size = config.n_embd
gpt2_model = P2GPTBlock(config)
gpt2_model.load_state_dict(gpt2_model_pretrained.state_dict(), strict=False)
gpt2_model.eval()

# Define a language modeling head (linear layer that maps hidden state -> vocab)
lm_head = nn.Linear(hidden_size, vocab_size, bias=False)
lm_head.eval()

# Tie weights if desired (like original GPT2LMHeadModel)
lm_head.weight = gpt2_model.wte.weight  # Tie to input embedding weights

# Input text
text = "Ive been fan of the"
input_ids = tokenizer.encode(text, return_tensors='pt')  # shape: [1, seq_len]

# Get hidden states from GPT2Model
with torch.no_grad():
    outputs = gpt2_model(input_ids)
    last_hidden_states = outputs.last_hidden_state  # shape: [1, seq_len, hidden_size]

# Get the hidden state of the last token
last_token_hidden = last_hidden_states[:, -1, :]  # shape: [1, hidden_size]

# Pass through the LM head to get logits over vocab
logits = lm_head(last_token_hidden)  # shape: [1, vocab_size]

# Predict next token
next_token_id = torch.argmax(logits, dim=-1).item()
next_token = tokenizer.decode([next_token_id])

print(f"Next token: '{next_token}'")


Next token: ' game'


In [83]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Block
import torch

# Load pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

# Input text
input_text = "Ive been fan of the game for a long time and I'm sure I'll be playing it again. I'm sure I'll be playing it again."
input_ids = tokenizer.encode(input_text, return_tensors='pt')  # shape: [1, seq_len]

# Predict logits for next token
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits  # shape: [1, seq_len, vocab_size]

# Get logits for the last token in the sequence
last_token_logits = logits[:, -1, :]  # shape: [1, vocab_size]

# Sample or take argmax for prediction
predicted_token_id = torch.argmax(last_token_logits, dim=-1).item()
predicted_token = tokenizer.decode([predicted_token_id])

print(f"Next token: '{predicted_token}'")


ImportError: cannot import name 'GPT2Block' from 'transformers' (c:\dev\aiimgdetect\.venv\Lib\site-packages\transformers\__init__.py)